In [1]:
import syft as sy
from syft.spdz import spdz
from syft.mpc.securenn import decompose, select_shares, private_compare, generate_zero_shares_communication, msb, relu_deriv
from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor, _SPDZTensor, _SNNTensor
from syft.core.frameworks.torch import utils as torch_utils

import unittest
import numpy as np
import torch
import importlib

global display_chain

class Chain():
    def __init__(self, leaf=False):
        if not leaf:
            self.tensor = Chain(True)
            self.var = Chain(True)

display_chain = Chain()

display_chain.tensor.local = 'FloatTensor > _LocalTensor'

display_chain.tensor.pointer = 'FloatTensor > _PointerTensor'

display_chain.tensor.fixp_local = 'FloatTensor > _FixedPrecisionTensor > LongTensor > _LocalTensor'

display_chain.tensor.fixp_mpc_gpt = 'FloatTensor > _FixedPrecisionTensor > LongTensor > _SNNTensor > LongTensor > _GeneralizedPointerTensor'

display_chain.var.local = 'Variable > _LocalTensor\n' \
                          ' - FloatTensor > _LocalTensor\n' \
                          ' - - Variable > _LocalTensor\n' \
                          '   - FloatTensor > _LocalTensor'

display_chain.var.pointer = 'Variable > _PointerTensor\n' \
                            ' - FloatTensor > _PointerTensor\n' \
                            ' - - Variable > _PointerTensor\n' \
                            '   - FloatTensor > _PointerTensor'

display_chain.var.fixp_local = 'Variable > _FixedPrecisionTensor > Variable > _LocalTensor\n' \
                               ' - FloatTensor > _FixedPrecisionTensor > LongTensor > _LocalTensor\n' \
                               ' - - Variable > _FixedPrecisionTensor > Variable > _LocalTensor\n' \
                               '   - FloatTensor > _FixedPrecisionTensor > LongTensor > _LocalTensor'

display_chain.var.fixp_mpc_gpt = 'Variable > _FixedPrecisionTensor > Variable > _SNNTensor > Variable > _GeneralizedPointerTensor\n' \
                                 ' - FloatTensor > _FixedPrecisionTensor > LongTensor > _SNNTensor > LongTensor > _GeneralizedPointerTensor\n' \
                                 ' - - Variable > _FixedPrecisionTensor > Variable > _SNNTensor > Variable > _GeneralizedPointerTensor\n' \
                                 '   - FloatTensor > _FixedPrecisionTensor > LongTensor > _SNNTensor > LongTensor > _LocalTensor'


In [2]:
hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)

me.add_workers([bob, alice])
bob.add_workers([me, alice])
alice.add_workers([me, bob])

In [3]:
L = spdz.Q_BITS

def random_as(tensor, mod=L):
    r = torch.LongTensor(tensor.get_shape()).random_(mod)
    return r

def xor(x, y):
    return x + y - 2 * x * y

def flip(x, dim):
    indices = torch.arange(x.get_shape()[dim] - 1, -1, -1).long()

    if hasattr(x.child, 'pointer_tensor_dict'):
        indices = indices.send(*list(x.child.pointer_tensor_dict.keys()))

    return x.index_select(dim, indices)

In [28]:
x = (torch.LongTensor([[-1,3,-5,7],[-1,0,1,2]])).share(alice,bob)

In [30]:
%%timeit
out = x.relu()

1 loop, best of 3: 227 ms per loop


In [6]:
out.get()


 0  3  0  7
 0  0  1  2
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]